In [3]:
import pandas as pd
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import numpy as np
from sklearn import datasets

In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [5]:

# Importação GridSearchCV.
from sklearn.model_selection import GridSearchCV

In [6]:
#from google.colab import files
#uploaded = files.upload()

In [7]:
# Dados de Câncer de mama.
cancer = datasets.load_breast_cancer()

# Criação do dataset features e vetor labels.
features = cancer.data
labels = cancer.target

In [8]:
# Só para evitar a aparição dos warnings.
import warnings
warnings.filterwarnings('ignore')


Classifier
Adota-se como algoritmo para ajustar os parâmetros o AdaBoost.


In [9]:
# Sem nada dentro, pois vamos "variar" os parâmetros.
clf = DecisionTreeClassifier(random_state=0)  
iris = load_iris()



In [10]:
cross_val_score(clf, iris.data, iris.target, cv=10)

array([1.        , 0.93333333, 1.        , 0.93333333, 0.93333333,
       0.86666667, 0.93333333, 1.        , 1.        , 1.        ])

Criação de Combinações de Parâmetros
Com base numa “lista”” de parâmetros o GridSearchCV criará as combinações e depois as avaliará, por exemplo, quero testar alguns valores de parâmetros de um AdaBoost. Os valores desta “lista” foram arbitrariamente escolhidos e sem nenhum critério técnico, o objetivo é encontrar valores otimizados.



In [12]:

# Exemplo dos parâmetros que quero testar.
parametros = { 'criterion':['entropy','gini'],
              'max_depth':[20,25]}

In [14]:
# Criação do objeto do GridSearchCV.
grid = GridSearchCV(estimator = clf,          # É a nossa Arvore de Classificação.
                    param_grid = parametros,  # É aquele dicionário com valores para serem testados.
                    scoring = 'f1',           # Arbitrariamente escolhi o f1, adiante explico com detalhes.
                    cv = 20)                  # Idem, arbitratiamente escolhi 20 e adiante será explanado.

In [15]:
#Após a criação do objeto do GridSearchCV, pode-se treinar e imprimir os resultados.

# Treinando o grid.
grid.fit(features, labels)

# Imprimindo os resultados.
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,split10_test_score,split11_test_score,split12_test_score,split13_test_score,split14_test_score,split15_test_score,split16_test_score,split17_test_score,split18_test_score,split19_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010150,0.000705,0.000816,0.000064,entropy,20,"{'criterion': 'entropy', 'max_depth': 20}",0.944444,0.941176,0.944444,0.820513,0.944444,0.947368,0.914286,0.972973,0.947368,1.000000,0.971429,0.875000,0.918919,0.944444,0.972973,0.944444,0.914286,1.000000,0.944444,0.972973,0.941797,0.039926,1
1,0.010312,0.000683,0.000828,0.000056,entropy,25,"{'criterion': 'entropy', 'max_depth': 25}",0.944444,0.941176,0.944444,0.820513,0.944444,0.947368,0.914286,0.972973,0.947368,1.000000,0.971429,0.875000,0.918919,0.944444,0.972973,0.944444,0.914286,1.000000,0.944444,0.972973,0.941797,0.039926,1
2,0.008599,0.000854,0.000950,0.000478,gini,20,"{'criterion': 'gini', 'max_depth': 20}",0.947368,0.914286,0.914286,0.857143,0.944444,0.944444,0.823529,0.947368,0.944444,0.944444,0.941176,0.903226,0.918919,0.944444,0.944444,0.944444,0.971429,0.941176,1.000000,0.909091,0.930005,0.036976,3
3,0.008609,0.000904,0.000839,0.000132,gini,25,"{'criterion': 'gini', 'max_depth': 25}",0.947368,0.914286,0.914286,0.857143,0.944444,0.944444,0.823529,0.947368,0.944444,0.944444,0.941176,0.903226,0.918919,0.944444,0.944444,0.944444,0.971429,0.941176,1.000000,0.909091,0.930005,0.036976,3


Há muitas colunas nesse resultado, então vamos fazer um subset escolhendo as colunas de interesse.

In [16]:
# Imprime colunas de interesse.
pd.DataFrame(grid.cv_results_)[['params','rank_test_score','mean_test_score']]

,params,rank_test_score,mean_test_score
0,"{'criterion': 'entropy', 'max_depth': 20}",1,0.941797
1,"{'criterion': 'entropy', 'max_depth': 25}",1,0.941797
2,"{'criterion': 'gini', 'max_depth': 20}",3,0.930005
3,"{'criterion': 'gini', 'max_depth': 25}",3,0.930005


Conforme a tabela acima, o melhor resultado de f1 é produzido pelos parâmetros {'criterion': entropy, 'max_depth': 20}. O atributo .best_params_ pode ser usado também para obter os melhores parâmetros.

In [17]:
# Imprime os parâmetros que produziram o ".best_score_".
grid.best_params_

{'criterion': 'entropy', 'max_depth': 20}

Bem como é possível imprimir o valor de f1 usando o atributo .best_score_.

In [18]:
# Imprimindo o score.
grid.best_score_

0.941796531885541

***Cross Validation***

*Uma pequena revisão.*

O que é Cross Validation?
É o uso de um dataset independente para avaliar o grau e a capacidade de generalização do seu modelo.

Note que o objeto grid foi definido com cv = 20 e quando cv é um número inteiro o GridSearchCV executa um StratifiedKFolds, isso quer dizer que o dataset foi divido em 20 partes (ou folds) e cada parte foi usada como test em uma simulação.

In [19]:
# Exemplo para o uso do StratifiedKFolds com 5 folds.
grid_2 = GridSearchCV(estimator = clf,
                      param_grid = parametros,
                      cv = 5,
                      scoring = 'f1')

# Imprime o f1
grid_2.fit(features,labels).best_score_


0.9483386869331364

O cv também aceita a atribuição de um objeto do StratifiedShuffleSplit, neste caso é necessário a definição da quantidade de n_splits e o tamanho do dataset de testes.

In [20]:
# Exemplo para o uso do StratifiedShuffleSplit.

# Importação do módulo do StratifiedShuffleSplit.
from sklearn.model_selection import StratifiedShuffleSplit

# Usando o constructor para criar o objeto sss
sss = StratifiedShuffleSplit(n_splits = 20,       # 20 simulações.
                             test_size = 0.2,     # 20% do dataset será de testes.
                             random_state = 42)   # Permitir a reprodutibilidade.

# Criando um objeto do GridSearchCV
grid_3 = GridSearchCV(estimator = clf,
                      param_grid = parametros,
                      cv = sss,
                      scoring = 'f1')

# Imprime o f1
grid_3.fit(features,labels).best_score_

0.945398037061391

Por fim, há a opção de não usar o cv do GridSearchCV, isto seria feito omitindo-o.

In [21]:
# Exemplo do GridSearchCV sem cv. Demanda um cv manual.

# Criando um objeto do GridSearchCV sem cv.
grid_4 = GridSearchCV(estimator = clf,
                      param_grid = parametros,
                      scoring = 'f1')

# Imprime o f1
grid_4.fit(features,labels).best_score_

0.9483386869331364

Note que nessas alternativas de cross validation o objetivo é usar métricas para a escolha do modelo que não sejam superestimadas, evitando assim o problema de overfitting.

Scoring
> Cada simulação terá como base de avaliação o scoring, e a configuração básica seria a definição de uma das métricas:

>recall;
>precision;
>accuracy, e/ou;
>f1 ou fbeta_score.

Logo, os resultados apresentados no atributo .cv_results_ cujas colunas possuam o sufixo _score referem-se ao scoring definido, para os exemplos anteriores usou-se o f1.

Contudo, há casos onde se necessita a avaliação de não só de um parâmetro, mas de vários. Dessa maneira, introduz-se o make_scorer (leia a documentação do make_scorer), com esse módulo é possível fazer com que o GridSearchCV calcule diversos parâmetros.

In [22]:
# Importando o Make Scorer
from sklearn.metrics import make_scorer

# Importando os módulos de cálculo de métricas
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import fbeta_score

# Criando um dicionário com as métricas que desejo calcular.
meus_scores = {'accuracy' :make_scorer(accuracy_score),
               'recall'   :make_scorer(recall_score),
               'precision':make_scorer(precision_score),
               'f1'       :make_scorer(fbeta_score, beta = 1)}

# Exemplo para o uso scoring igual ao meus_scores.
grid_5 = GridSearchCV(estimator = clf,
                      param_grid = parametros,
                      cv = 5,
                      scoring = meus_scores,   # É o meus_scores
                      refit = 'f1')            # Observe que foi configurado para f1

Espera-se que o grid_5 possua novas colunas das quais tenha sufixos accuracy, recall, precision e f1. Vamos imprimir os nomes das colunas.

O GridSearchCV é uma ferramenta que automatiza muito das etapas repetitivas do processo de tuning, contudo há diversas peculariedades no uso dela que a tornam um pouco traiçoeira.

É preciso ter em mente que o refit se faz necessário quando se usa o make_scorer pode não levar em consideração todas as métricas para selecionar a melhor solução, sendo assim só a métrica definida no refit usada para eligir os melhores parâmetros.